In [47]:
import gensim
import re
import pandas as pd
import numpy as np
from langdetect import detect
import pyarabic.araby as araby
import nltk

from nltk.stem.isri import ISRIStemmer
from nltk import word_tokenize
from nltk import word_tokenize


In [48]:
df = pd.read_csv('ar_reviews_100k.csv')
print(df)

          label                                               text
0      Positive  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1      Positive  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2      Positive  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3      Positive  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4      Positive  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...         ...                                                ...
99994  Negative  معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995  Negative  لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996  Negative  كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997  Negative  مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...
99998  Negative  لن ارجع إليه مرة اخرى . قربه من البحر. المكان ...

[99999 rows x 2 columns]


In [49]:
df["label"]=df["label"].str.replace("Positive","1")
df["label"]=df["label"].str.replace("Mixed","0")
df["label"]=df["label"].str.replace("Negative","-1")
print(df)

      label                                               text
0         1  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1         1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2         1  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3         1  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4         1  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...     ...                                                ...
99994    -1  معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995    -1  لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996    -1  كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997    -1  مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...
99998    -1  لن ارجع إليه مرة اخرى . قربه من البحر. المكان ...

[99999 rows x 2 columns]


In [50]:
def get_unique(df,colname):
  sentences_arr=df[colname].to_numpy() #convert df to np array
  words=' '.join(sentences_arr).split(' ') # join all strings into one string then splitting to get the words
  words=np.array(words) #convert to np array
  unique,counts = np.unique(words,return_counts=True)
  return unique, counts


def get_unique_dict(unique,counts):
  unique_dict=dict(zip(unique,counts))
  unique_dict=dict(sorted(unique_dict.items(), key=lambda item: item[1],reverse=True))
  return unique_dict


def remove_redundant_words(unique_dict):
  stop_words=['من','على','عن','في','فى','و','ان','هذا','او','كتب','...','.','','الى','فيه','انه','قبل','//','..','،',':',"؟",'/']
  for k, v in list(unique_dict.items()):
      if(unique_dict[k]<100 or k in stop_words):
        del unique_dict[k]

  return unique_dict


def remove_useless_words(sentence):
  res=[]
  for word in word_tokenize(sentence):
    if word in unique_dict:
      res.append(word)
  return res

In [51]:
unique,counts=get_unique(df,"text")
unique_dict=get_unique_dict(unique,counts)
unique_dict=remove_redundant_words(unique_dict)

df["text"]=df["text"].apply(remove_useless_words)

In [52]:
def get_freq(words):
  freq=dict()
  for word in words:
      if word in freq:
        freq[word]+=1
      else:
        freq[word]=1
  return freq

In [53]:
df_freq=df.copy()
df_freq["freq"]=df["text"].apply(get_freq)


In [54]:
df_freq

,label,text,freq
0,1,"[ممتاز, نوعا, ما, النظافة, والموقع, المطعم]","{'ممتاز': 1, 'نوعا': 1, 'ما': 1, 'النظافة': 1,..."
1,1,"[أحد, أسباب, نجاح, أن, كل, شخص, هذه, الدولة, ن...","{'أحد': 1, 'أسباب': 1, 'نجاح': 1, 'أن': 1, 'كل..."
2,1,"[القاهرة, هدوء, حقيقة, ما, تلك, البلاد, بحق, ا...","{'القاهرة': 1, 'هدوء': 1, 'حقيقة': 1, 'ما': 1,..."
3,1,"[مبدئيا, اللي, زي, الفيل, الازرق, احسن, احمد, ...","{'مبدئيا': 1, 'اللي': 1, 'زي': 2, 'الفيل': 1, ..."
4,1,"[جزء, لا, دبي, فندق, متكامل, الخدمات, مريح, نف...","{'جزء': 1, 'لا': 2, 'دبي': 1, 'فندق': 1, 'متكا..."
...,...,...,...
99994,-1,"[معرفش, ليه, كنت, أكملها, وهي, مش, البداية, ال...","{'معرفش': 1, 'ليه': 1, 'كنت': 1, 'أكملها': 1, ..."
99995,-1,"[لا, يستحق, يكون, بوكنق, لانه, سيئ, لا, شي, لا...","{'لا': 3, 'يستحق': 1, 'يكون': 1, 'بوكنق': 1, '..."
99996,-1,"[كتاب, ضعيف, جدا, ولم, استمتع, به, كل, قصه, سر...","{'كتاب': 1, 'ضعيف': 1, 'جدا': 1, 'ولم': 1, 'اس..."
99997,-1,"[مملة, جدا, محمد, حسن, علوان, والوصف, دقيق, حد...","{'مملة': 1, 'جدا': 1, 'محمد': 1, 'حسن': 1, 'عل..."


In [55]:
model = gensim.models.Word2Vec.load('full_uni_sg_300_wiki.mdl')

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [56]:
df_copy=df_freq
df_copy=df_copy[(df_copy['text'].str.len() != 0)]

In [57]:
df_copy.reset_index(inplace=True,drop=True)
df_copy["text"][58]

['رائعة', 'بس', 'قرأتها', 'احداث', 'البلد', 'الان', 'أمر']

In [58]:
def wordExists(word):
    try:
        x = model.wv[word]
        return True
    except KeyError:
        return False

In [59]:
arrOfEmbeddings = []
for key in unique_dict.keys():
    key = clean_str(key)
    if wordExists(key):
        arrOfEmbeddings.append(model.wv[key])
    else:
        arrOfEmbeddings.append(np.zeros(300))
len(arrOfEmbeddings)

5691

In [60]:
df_copy

,label,text,freq
0,1,"[ممتاز, نوعا, ما, النظافة, والموقع, المطعم]","{'ممتاز': 1, 'نوعا': 1, 'ما': 1, 'النظافة': 1,..."
1,1,"[أحد, أسباب, نجاح, أن, كل, شخص, هذه, الدولة, ن...","{'أحد': 1, 'أسباب': 1, 'نجاح': 1, 'أن': 1, 'كل..."
2,1,"[القاهرة, هدوء, حقيقة, ما, تلك, البلاد, بحق, ا...","{'القاهرة': 1, 'هدوء': 1, 'حقيقة': 1, 'ما': 1,..."
3,1,"[مبدئيا, اللي, زي, الفيل, الازرق, احسن, احمد, ...","{'مبدئيا': 1, 'اللي': 1, 'زي': 2, 'الفيل': 1, ..."
4,1,"[جزء, لا, دبي, فندق, متكامل, الخدمات, مريح, نف...","{'جزء': 1, 'لا': 2, 'دبي': 1, 'فندق': 1, 'متكا..."
...,...,...,...
99641,-1,"[معرفش, ليه, كنت, أكملها, وهي, مش, البداية, ال...","{'معرفش': 1, 'ليه': 1, 'كنت': 1, 'أكملها': 1, ..."
99642,-1,"[لا, يستحق, يكون, بوكنق, لانه, سيئ, لا, شي, لا...","{'لا': 3, 'يستحق': 1, 'يكون': 1, 'بوكنق': 1, '..."
99643,-1,"[كتاب, ضعيف, جدا, ولم, استمتع, به, كل, قصه, سر...","{'كتاب': 1, 'ضعيف': 1, 'جدا': 1, 'ولم': 1, 'اس..."
99644,-1,"[مملة, جدا, محمد, حسن, علوان, والوصف, دقيق, حد...","{'مملة': 1, 'جدا': 1, 'محمد': 1, 'حسن': 1, 'عل..."


In [61]:
#def parallelEmb (lister):
    #for index in range(df_copy.shape[0]):
#    sentence = np.array(lister)
#    print (sentence)
#    #arr=[] # array of vectors of words in a single sentence
#    arr= [arrOfEmbeddings[ind] for ind in range(len(sentence)) if sentence[ind] in unique_dict.keys() ]
#    if len(arr) ==0:
#        arr = list(np.zeros(100))
#        #print(type(arr))
#    arr=np.array(arr)
#    avg=np.average(arr, axis=0)
#    if type(avg) == np.float64:
#        avg = list(np.zeros(100))
#        #print(type(arr))
#    #print("done")
#    return avg

In [62]:
#dftemp = df_freq.head()
#df22=dftemp["text"].apply(parallelEmb)
#df22[0]

In [63]:
#global count
#count = 0
def parallelEmb1 (freqDict):#freqDict
    #for index in range(df_copy.shape[0]):
    sentence = np.array(list(freqDict.keys()))
    #print(freqDict,"\n____________________")
    #print(sentence)
    global count
    #print(count)
    #count = count + 1
    #arr=[] # array of vectors of words in a single sentence
    arr= [arrOfEmbeddings[ind]*freqDict[sentence[ind]]
          for ind in range(len(sentence)) 
          if sentence[ind] in unique_dict.keys()]
    totalFreq = sum(freqDict.values())
    if len(arr) ==0:
        arr = list(np.zeros(300))
        #print(type(arr))
    arr=np.array(arr)
    #avg=np.average(arr,axis=0)
    vectorSum = np.sum(arr,axis=0)
    #return arr
    #vec=np.sum(arr, axis=0)
    #if type(avg) == np.float64:
        #avg = list(np.zeros(100))
        #print(type(arr))
    #print(len(avg))
    #print("done")
    if (totalFreq == 0):
        vectorSum = list(np.zeros(300))
    else: vectorSum = vectorSum * 1/totalFreq
    return vectorSum

In [64]:
df2 = df_copy#.head()
df22=df2["freq"].apply(parallelEmb1)
df22

0        [-0.11026106, 0.0046779285, 0.121830255, 0.046...
1        [-0.1869652, 0.03043485, 0.124365196, 0.043905...
2        [-0.18024886, 0.053632904, 0.115090765, 0.0120...
3        [-0.16312025, 0.016828109, 0.13799354, 0.05673...
4        [-0.1251224, 0.042683393, 0.0631845, 0.0656028...
                               ...                        
99641    [-0.17478272, 0.028179262, 0.1462719, 0.030914...
99642    [-0.17779732, 0.022778112, 0.13924994, 0.02736...
99643    [-0.15548651, 0.020547757, 0.10377676, 0.04795...
99644    [-0.18526174, 0.019239599, 0.1419204, 0.044390...
99645    [-0.17646493, 0.024540586, 0.12405683, 0.02704...
Name: freq, Length: 99646, dtype: object

In [65]:
#print((df22[620]))
#len(df22[620])

In [68]:
df222=df22
names = list(range(1,301))
dfOut = pd.DataFrame()
dfOut[names] = pd.DataFrame(df222.tolist())
dfOut

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut[names] = pd.DataFrame(df222.tolist())
C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\4014946585.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,-0.110261,0.004678,0.121830,0.046323,-0.059155,-0.130254,-0.090927,0.033228,-0.109235,-0.024638,...,0.106310,0.021777,-0.051642,-0.115915,-0.125819,0.053349,0.014209,-0.060097,0.002827,0.010759
1,-0.186965,0.030435,0.124365,0.043906,-0.057767,-0.216118,-0.105242,0.019484,-0.103281,-0.034242,...,0.042305,-0.047830,0.000810,-0.144324,-0.054567,0.043735,0.059880,-0.030497,0.006474,-0.054189
2,-0.180249,0.053633,0.115091,0.012021,-0.072666,-0.140080,-0.068927,0.020016,-0.099535,-0.003294,...,0.043941,-0.011103,-0.020388,-0.127930,-0.073511,0.079643,0.073929,-0.041899,-0.015371,-0.030381
3,-0.163120,0.016828,0.137994,0.056731,-0.066900,-0.220143,-0.111483,0.003781,-0.117558,-0.022372,...,0.049491,-0.011577,-0.001263,-0.150690,-0.057700,0.022575,0.057661,-0.033550,0.003368,-0.074426
4,-0.125122,0.042683,0.063184,0.065603,-0.121049,-0.174491,-0.151510,0.066725,-0.082309,0.025271,...,0.122178,-0.017012,-0.049806,-0.137376,-0.054088,0.042490,0.060064,-0.086987,-0.016922,-0.056258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99641,-0.174783,0.028179,0.146272,0.030914,-0.089677,-0.165285,-0.085900,0.045137,-0.101399,-0.004959,...,0.052644,-0.021606,-0.012585,-0.135240,-0.063006,0.083830,0.078058,-0.043372,0.003859,-0.033853
99642,-0.177797,0.022778,0.139250,0.027364,-0.100054,-0.186798,-0.127894,0.011642,-0.114786,0.009217,...,0.104184,-0.006060,-0.000830,-0.160489,-0.064464,0.077569,0.064476,-0.051492,-0.007229,-0.025148
99643,-0.155487,0.020548,0.103777,0.047959,-0.087139,-0.191943,-0.112236,0.031066,-0.115495,0.018672,...,0.081337,-0.012110,0.001530,-0.157777,-0.069630,0.060375,0.072171,-0.031370,-0.018294,-0.051415
99644,-0.185262,0.019240,0.141920,0.044391,-0.098279,-0.190461,-0.108721,0.030528,-0.105445,-0.011330,...,0.047457,-0.036559,-0.009514,-0.139336,-0.067670,0.082793,0.066353,-0.031775,0.003938,-0.022624


In [69]:
dfOut["label"] = df["label"]

C:\Users\AUC\AppData\Local\Temp\ipykernel_16388\2804044779.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfOut["label"] = df["label"]


In [70]:
dfOut

,1,2,3,4,5,6,7,8,9,10,...,292,293,294,295,296,297,298,299,300,label
0,-0.110261,0.004678,0.121830,0.046323,-0.059155,-0.130254,-0.090927,0.033228,-0.109235,-0.024638,...,0.021777,-0.051642,-0.115915,-0.125819,0.053349,0.014209,-0.060097,0.002827,0.010759,1
1,-0.186965,0.030435,0.124365,0.043906,-0.057767,-0.216118,-0.105242,0.019484,-0.103281,-0.034242,...,-0.047830,0.000810,-0.144324,-0.054567,0.043735,0.059880,-0.030497,0.006474,-0.054189,1
2,-0.180249,0.053633,0.115091,0.012021,-0.072666,-0.140080,-0.068927,0.020016,-0.099535,-0.003294,...,-0.011103,-0.020388,-0.127930,-0.073511,0.079643,0.073929,-0.041899,-0.015371,-0.030381,1
3,-0.163120,0.016828,0.137994,0.056731,-0.066900,-0.220143,-0.111483,0.003781,-0.117558,-0.022372,...,-0.011577,-0.001263,-0.150690,-0.057700,0.022575,0.057661,-0.033550,0.003368,-0.074426,1
4,-0.125122,0.042683,0.063184,0.065603,-0.121049,-0.174491,-0.151510,0.066725,-0.082309,0.025271,...,-0.017012,-0.049806,-0.137376,-0.054088,0.042490,0.060064,-0.086987,-0.016922,-0.056258,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99641,-0.174783,0.028179,0.146272,0.030914,-0.089677,-0.165285,-0.085900,0.045137,-0.101399,-0.004959,...,-0.021606,-0.012585,-0.135240,-0.063006,0.083830,0.078058,-0.043372,0.003859,-0.033853,-1
99642,-0.177797,0.022778,0.139250,0.027364,-0.100054,-0.186798,-0.127894,0.011642,-0.114786,0.009217,...,-0.006060,-0.000830,-0.160489,-0.064464,0.077569,0.064476,-0.051492,-0.007229,-0.025148,-1
99643,-0.155487,0.020548,0.103777,0.047959,-0.087139,-0.191943,-0.112236,0.031066,-0.115495,0.018672,...,-0.012110,0.001530,-0.157777,-0.069630,0.060375,0.072171,-0.031370,-0.018294,-0.051415,-1
99644,-0.185262,0.019240,0.141920,0.044391,-0.098279,-0.190461,-0.108721,0.030528,-0.105445,-0.011330,...,-0.036559,-0.009514,-0.139336,-0.067670,0.082793,0.066353,-0.031775,0.003938,-0.022624,-1


In [71]:
#dfOut.to_csv('final3rdEmbedding.csv', encoding = 'utf-8-sig',index=False) 

In [72]:
#df = pd.read_csv('final3rdEmbedding.csv')
df=dfOut
df

,1,2,3,4,5,6,7,8,9,10,...,292,293,294,295,296,297,298,299,300,label
0,-0.110261,0.004678,0.121830,0.046323,-0.059155,-0.130254,-0.090927,0.033228,-0.109235,-0.024638,...,0.021777,-0.051642,-0.115915,-0.125819,0.053349,0.014209,-0.060097,0.002827,0.010759,1
1,-0.186965,0.030435,0.124365,0.043906,-0.057767,-0.216118,-0.105242,0.019484,-0.103281,-0.034242,...,-0.047830,0.000810,-0.144324,-0.054567,0.043735,0.059880,-0.030497,0.006474,-0.054189,1
2,-0.180249,0.053633,0.115091,0.012021,-0.072666,-0.140080,-0.068927,0.020016,-0.099535,-0.003294,...,-0.011103,-0.020388,-0.127930,-0.073511,0.079643,0.073929,-0.041899,-0.015371,-0.030381,1
3,-0.163120,0.016828,0.137994,0.056731,-0.066900,-0.220143,-0.111483,0.003781,-0.117558,-0.022372,...,-0.011577,-0.001263,-0.150690,-0.057700,0.022575,0.057661,-0.033550,0.003368,-0.074426,1
4,-0.125122,0.042683,0.063184,0.065603,-0.121049,-0.174491,-0.151510,0.066725,-0.082309,0.025271,...,-0.017012,-0.049806,-0.137376,-0.054088,0.042490,0.060064,-0.086987,-0.016922,-0.056258,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99641,-0.174783,0.028179,0.146272,0.030914,-0.089677,-0.165285,-0.085900,0.045137,-0.101399,-0.004959,...,-0.021606,-0.012585,-0.135240,-0.063006,0.083830,0.078058,-0.043372,0.003859,-0.033853,-1
99642,-0.177797,0.022778,0.139250,0.027364,-0.100054,-0.186798,-0.127894,0.011642,-0.114786,0.009217,...,-0.006060,-0.000830,-0.160489,-0.064464,0.077569,0.064476,-0.051492,-0.007229,-0.025148,-1
99643,-0.155487,0.020548,0.103777,0.047959,-0.087139,-0.191943,-0.112236,0.031066,-0.115495,0.018672,...,-0.012110,0.001530,-0.157777,-0.069630,0.060375,0.072171,-0.031370,-0.018294,-0.051415,-1
99644,-0.185262,0.019240,0.141920,0.044391,-0.098279,-0.190461,-0.108721,0.030528,-0.105445,-0.011330,...,-0.036559,-0.009514,-0.139336,-0.067670,0.082793,0.066353,-0.031775,0.003938,-0.022624,-1


In [73]:
#
# Trying Different Models Here
#

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import sklearn.preprocessing as preprocess

In [75]:
df_remLabel=df.drop(columns=["label"])
df_label=df["label"]
x_train, x_test, y_train, y_test = ms.train_test_split(df_remLabel,df_label, test_size=0.2, random_state=1234)

In [76]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
#Logreg = LogisticRegression(max_iter=500, solver = 'newton-cg', n_jobs=8).fit(x_train_fit,y_train)
Logreg = LogisticRegression(random_state=0, max_iter=2000).fit(x_train,y_train)
Logreg_pred = Logreg.predict(x_test)
print(classification_report(y_test, Logreg_pred))

              precision    recall  f1-score   support

          -1       0.36      0.30      0.33      6604
           0       0.35      0.33      0.34      6644
           1       0.37      0.45      0.41      6682

    accuracy                           0.36     19930
   macro avg       0.36      0.36      0.36     19930
weighted avg       0.36      0.36      0.36     19930



In [77]:
from sklearn.tree import DecisionTreeClassifier
DT_clf = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)
DT_pred = DT_clf.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, DT_pred))

              precision    recall  f1-score   support

          -1       0.34      0.22      0.27      6604
           0       0.34      0.28      0.30      6644
           1       0.35      0.54      0.43      6682

    accuracy                           0.35     19930
   macro avg       0.35      0.35      0.33     19930
weighted avg       0.35      0.35      0.33     19930



In [78]:
from sklearn.naive_bayes import GaussianNB
NV_clf = GaussianNB().fit(x_train, y_train)
NV_pred = NV_clf.predict(x_test)
print(classification_report(y_test, NV_pred))

              precision    recall  f1-score   support

          -1       0.27      0.01      0.01      6604
           0       0.34      0.72      0.46      6644
           1       0.38      0.32      0.34      6682

    accuracy                           0.35     19930
   macro avg       0.33      0.35      0.27     19930
weighted avg       0.33      0.35      0.27     19930



In [79]:
from sklearn.neural_network import MLPClassifier
NN_clf_stop = MLPClassifier(random_state=1, max_iter=30, hidden_layer_sizes=(16,16), tol=1e-5, early_stopping=True, learning_rate_init=0.01)
NN_clf_stop.fit(x_train, y_train)
NN_pred_stop = NN_clf_stop.predict(x_test)
print(classification_report(y_test, NN_pred_stop))

              precision    recall  f1-score   support

          -1       0.34      0.63      0.44      6604
           0       0.34      0.01      0.01      6644
           1       0.37      0.43      0.40      6682

    accuracy                           0.35     19930
   macro avg       0.35      0.35      0.28     19930
weighted avg       0.35      0.35      0.28     19930



In [80]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
SGD_SVM_clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=500, tol=1e-3))
SGD_SVM_clf.fit(x_train, y_train)
SGD_SVM_pred = SGD_SVM_clf.predict(x_test)
print(classification_report(y_test, SGD_SVM_pred))

              precision    recall  f1-score   support

          -1       0.33      0.11      0.17      6604
           0       0.34      0.55      0.42      6644
           1       0.37      0.38      0.37      6682

    accuracy                           0.35     19930
   macro avg       0.35      0.35      0.32     19930
weighted avg       0.35      0.35      0.32     19930



In [81]:
#from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#vectorizer = TfidfVectorizer()